# 03 ANTs deform coregistration IntraSubject-InterTemporal between MRIdiagnosis and MRIrecurrent

Marianne Hannisdal/Arvid Lundervold

Last updated: **2025-03-13**

Kernel: segment-glioma (Python 3.11.9)

In [1]:
%matplotlib inline
# This to be able to display figures and graphs within the notebook browser

import ants
import os
import os.path as op
import shutil
import subprocess as subp
import pathlib
import glob
import shutil
import string
from datetime import date
import warnings
import numpy as np
import pandas as pd
import nibabel as nib
from nibabel.viewers import OrthoSlicer3D
import scipy
from nilearn import image
from nilearn import plotting
from nilearn.plotting import plot_roi
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from nilearn.image.image import mean_img
from sklearn.cluster import KMeans
from nilearn.masking import apply_mask
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.ensemble import RandomForestClassifier
import IPython
from matplotlib.colors import ListedColormap
import warnings
warnings.filterwarnings("ignore")

In [2]:
%env FREESURFER_HOME=/usr/local/freesurfer
%env FSL_HOME=/usr/local/fsl

env: FREESURFER_HOME=/usr/local/freesurfer
env: FSL_HOME=/usr/local/fsl


In [3]:
home = os.path.expanduser('~')

SITE = '10'
TREE = '/usr/bin/tree'

MRICONVERT = '/usr/local/freesurfer/bin/mri_convert'
FSLREORIENT2STD = '/usr/local/fsl/bin/fslreorient2std'
FLIRT = '/usr/local/fsl/bin/flirt'
FSLMATHS = '/usr/local/fsl/bin/fslmaths'
EASYREG = '/usr/local/freesurfer/bin/mri_easyreg'


base_dir = f"{os.path.expanduser('~')}/prj/glioma_recurrence/glioma_recurrence/data/{SITE}"
nifti_dir = f"{base_dir}/nifti"   
segment_glioma_dir = f"{base_dir}/segment_glioma_1"
easyreg_dir = f"{base_dir}/easyreg_1"
registered_dir = f"{base_dir}/registered_1"


all_chns = ['T1', 'CT1', 'T2', 'FLAIR']
template = 'CT1'
inp_chns = [c for c in all_chns if c != template]  


colors = [(1, 0, 0), (0, 0, 1), (0, 1, 0)]  # R -> B -> G
n_bins = [2, 3, 6, 10, 100]  # Discretizes the interpolation into bins
cmap_name = 'my_cm'
cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins[0])

### check if all directories in easyreg_dir contains 4 files

In [4]:
for d in os.listdir(easyreg_dir):
    if len(os.listdir(f"{easyreg_dir}/{d}")) != 4:
        print(f"Directory {d} does not contain 4 files")

## Applying registration from MRI_recurrent to MRI_diagnosis (MRI_diagnosis is template, MRI_recurrent is moving)

In the below code, ANTsPy is used for image registration, which is the process of transforming different sets of data into one coordinate system. This is often necessary in the medical imaging field, where images from different modalities or taken at different times need to be aligned to perform further analysis.

Here's a step-by-step breakdown of how ANTsPy is used in the code:

1. **Image Reading**: The `ants.image_read()` function is used to read the fixed and moving images. The fixed image is the reference image to which the moving image will be aligned.

    ```python
    fixed_image = ants.image_read(fixed_image_path)
    moving_image = ants.image_read(moving_image_path)
    ```

2. **Image Registration**: The `ants.registration()` function is used to perform the registration. The `type_of_transform` parameter is set to `'SyN'`, which stands for Symmetric Normalization. This is a powerful, high-dimensional, diffeomorphic image registration algorithm.

    ```python
    transform = ants.registration(fixed_image, moving_image, type_of_transform='SyN')
    ```

3. **Applying Transforms**: The `ants.apply_transforms()` function is used to apply the calculated transformation to the moving image. This function takes the fixed image, the moving image, and the transformation list as arguments, and returns the moving image transformed into the space of the fixed image.

    ```python
    registered_CT1 = ants.apply_transforms(fixed=ants.image_read(ref_CT1_coreg), 
                                           moving=ants.image_read(exam_CT1_coreg),
                                           transformlist=transform['fwdtransforms'])
    ```

4. **Image Writing**: The `ants.image_write()` function is used to write the registered image to a file.

    ```python
    ants.image_write(registered_CT1, output_CT1_coreg)
    ```

This process is repeated for each exam and each modality, allowing all images to be aligned to the same reference image.

## ANTsPy with nearest neighbor interpolation for segment_glioma

In [6]:
exams_list_site = sorted([x for x in next(os.walk(easyreg_dir))[1]])
exams_list_site

['10_036_20150101', '10_036_20160529']

### Defining the list of exams that is undergoing transformation - per patient

In [7]:
# defining the exams list, where the first exam is the temolate and the second exam is moving
exams_list = [
 '10_036_20150101',
 '10_036_20160529',
]

# Performing the transformation

In [7]:
# Create a dictionary to group exams by subject
subject_exams = {}
for exam in exams_list:
    subject_id = exam[:6]
    if subject_id not in subject_exams:
        subject_exams[subject_id] = []
    subject_exams[subject_id].append(exam)

# Function to perform deformable registration
def deformable_registration(fixed_image_path, moving_image_path):
    # Load the images
    fixed_image = ants.image_read(fixed_image_path)
    moving_image = ants.image_read(moving_image_path)

    # Perform registration
    transform = ants.registration(fixed_image, moving_image, type_of_transform='SyN')

    return transform

# Loop over each subject
for subject_id, exams in subject_exams.items():
    # Sort exams by date to ensure the first date is the reference
    exams.sort()
    
    # Reference exam
    reference_exam = exams[0]
    ref_CT1_coreg = f"{easyreg_dir}/{reference_exam}/{reference_exam}_CT1_coreg.nii.gz"

    # Check if the reference image exists
    if not os.path.exists(ref_CT1_coreg):
        print(f"Reference image {ref_CT1_coreg} does not exist. Skipping this exam.")
        continue
    
    # Create a directory for the registered images if it doesn't exist
    subject_registered_dir = f"{registered_dir}/{subject_id}"
    os.makedirs(subject_registered_dir, exist_ok=True)
    
    # Loop over other exams and register them to the reference
    for exam in exams[1:]:
        exam_CT1_coreg = f"{easyreg_dir}/{exam}/{exam}_CT1_coreg.nii.gz"
        output_CT1_coreg = f"{subject_registered_dir}/{exam}_CT1_coreg_registered_to_{reference_exam}.nii.gz"

        # Check if the exam image exists
        if not os.path.exists(exam_CT1_coreg):
            print(f"Exam image {exam_CT1_coreg} does not exist. Skipping this exam.")
            continue

        # Perform deformable registration
        transform = deformable_registration(ref_CT1_coreg, exam_CT1_coreg)

        # Save the registered CT1 image
        registered_CT1 = ants.apply_transforms(
            fixed=ants.image_read(ref_CT1_coreg), 
            moving=ants.image_read(exam_CT1_coreg),
            transformlist=transform['fwdtransforms']
        )
        ants.image_write(registered_CT1, output_CT1_coreg)

        # Register other modalities using the same transformation
        modalities = ['T1', 'T2', 'FLAIR', 'synthseg', 'segmentation_native']
        for modality in modalities:
            if modality ==  'segmentation_native':
                moving_image_path = f"{segment_glioma_dir}/{exam}/segmentation_native_{exam}_CT1_coreg.nii.gz"
                output_path = f"{subject_registered_dir}/segmentation_native_{exam}_CT1_coreg_registered_to_{reference_exam}.nii.gz"
                interpolator = 'nearestNeighbor'
            else:
                moving_image_path = f"{easyreg_dir}/{exam}/{exam}_{modality}_coreg.nii.gz"
                output_path = f"{subject_registered_dir}/{exam}_{modality}_coreg_registered_to_{reference_exam}.nii.gz"
                interpolator = 'linear'  # Use linear interpolation for continuous data
            
            # Check if the modality file exists before applying the transformation
            if not os.path.exists(moving_image_path):
                print(f"Modality file {moving_image_path} does not exist. Skipping this modality.")
                continue
            
            # Apply the transformation to other modalities
            registered_image = ants.apply_transforms(
                fixed=ants.image_read(ref_CT1_coreg), 
                moving=ants.image_read(moving_image_path),
                transformlist=transform['fwdtransforms'],
                interpolator=interpolator  # Pass the appropriate interpolator here
            )
            ants.image_write(registered_image, output_path)
    
    # Copy the native segmentation of the reference exam into registered_dir/subject_id
    segmentation_native_path = f"{segment_glioma_dir}/{reference_exam}/segmentation_native_{reference_exam}_CT1_coreg.nii.gz"
    shutil.copy(segmentation_native_path, subject_registered_dir)

    # Copy the T1, T2, FLAIR, CT1 of the reference exam into registered_dir/subject_id
    for modality in ['T1', 'T2', 'FLAIR', 'CT1']:
        modality_path = f"{easyreg_dir}/{reference_exam}/{reference_exam}_{modality}_coreg.nii.gz"
        shutil.copy(modality_path, subject_registered_dir)

    # Once all registrations for a subject are done, move the processed exams to the registered directory
    for exam in exams:
        exam_source_dir = f"{easyreg_dir}/{exam}"
        exam_target_dir = f"{registered_dir}/{subject_id}/{exam}"
        
        if not os.path.exists(exam_target_dir):
            shutil.copytree(exam_source_dir, exam_target_dir)

print("Registration completed and exams moved to the registered directory.")


Registration completed and exams moved to the registered directory.


# Calculating the (transformed) volumes

In [8]:
exam1 = exams_list[0]
exam2 = exams_list[-1]

home = os.path.expanduser("~")
PRJ_DIR = f'{home}/prj/glioma_recurrence/glioma_recurrence/data/{SITE}'
REGISTERED_dir = f'{PRJ_DIR}/registered/{exam1[:6]}'

# exams_list is defined in the previous cell, performing the ANTS registration
print(exams_list)
print("Exam 1:", exam1)
print("Exam 2:", exam2)
print(f'{REGISTERED_dir}/{exam1}')

['10_036_20150101', '10_036_20160529']
Exam 1: 10_036_20150101
Exam 2: 10_036_20160529
/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20150101


In [9]:
%%bash -s "$exam1" "$exam2" "$REGISTERED_dir"

echo "exam1 (reference): $1"
echo "exam2 (moving): $2"
echo "REGISTERED_dir: $3"

# Define paths for synthseg segmentations
segmentation_moving="${3}/${2}_CT1_coreg_synthseg_parc_robust_registered_to_${1}.nii.gz"

# Vol file paths
vol_file_moving="${3}/${2}_CT1_coreg_synthseg_parc_robust_registered_to_${1}_vol.csv"

# Set up FreeSurfer environment
FREESURFER_HOME=/usr/local/freesurfer; export FREESURFER_HOME
export PATH=${FREESURFER_HOME}/bin:${PATH}
source ${FREESURFER_HOME}/SetUpFreeSurfer.sh

# Recalculate volumes for the registered moving segmentation
echo "Calculating volumes for moving: $segmentation_moving"
mri_segstats --seg $segmentation_moving --i $segmentation_moving --sum $vol_file_moving

# Output the paths to the vol files
echo "Volume calculations completed."
echo "Moving volume file: $vol_file_moving"


exam1 (reference): 10_036_20150101
exam2 (moving): 10_036_20160529
REGISTERED_dir: /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036
Calculating volumes for moving: /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20160529_CT1_coreg_synthseg_parc_robust_registered_to_10_036_20150101.nii.gz

7.4.1
cwd 
cmdline mri_segstats --seg /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20160529_CT1_coreg_synthseg_parc_robust_registered_to_10_036_20150101.nii.gz --i /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20160529_CT1_coreg_synthseg_parc_robust_registered_to_10_036_20150101.nii.gz --sum /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20160529_CT1_coreg_synthseg_parc_robust_registered_to_10_036_20150101_vol.csv 
sysname  Linux
hostname MariannePrecision7680
machine  x86_64
user     marianne
whitesurfname

## Display output in freeview

In [10]:
exam1 = exams_list[0]
exam2 = exams_list[-1]

home = os.path.expanduser("~")
PRJ_DIR = f'{home}/prj/glioma_recurrence/glioma_recurrence/data/{SITE}'
REGISTERED_dir = f'{PRJ_DIR}/registered/{exam1[:6]}'

# exams_list is defined in the previous cell, performing the ANTS registration
print(exams_list)
print("Exam 1:", exam1)
print("Exam 2:", exam2)
print(f'{REGISTERED_dir}/{exam1}')

['10_036_20150101', '10_036_20160529']
Exam 1: 10_036_20150101
Exam 2: 10_036_20160529
/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20150101


In [11]:
%%bash -s "$exam1" "$exam2" "$REGISTERED_dir"

echo "exam1 (reference): $1"
echo "exam2 (moving): $2"
echo "REGISTERED_DIR: $3"

# Print all paths to verify
echo "Checking paths..."
for modality in FLAIR T2 CT1 T1; do
    echo "${3}/${1}_${modality}_coreg.nii.gz"
    echo "${3}/${2}_${modality}_coreg_registered_to_${1}.nii.gz"
done

echo "${3}/${1}_CT1_synthseg_parc_robust.nii.gz"
echo "${3}/segmentation_native_${1}_CT1_coreg.nii.gz"
echo "${3}/${2}_CT1_coreg_synthseg_parc_robust_registered_to_${1}.nii.gz"
echo "${3}/segmentation_native_${2}_CT1_coreg_registered_to_${1}.nii.gz"

# Set up FreeSurfer and FSL environments
# Uncomment the MacOS line if running on a Mac
# FREESURFER_HOME=/Applications/freesurfer/7.4.1; export FREESURFER_HOME #MacOS
FREESURFER_HOME=/usr/local/freesurfer; export FREESURFER_HOME
export PATH=${FREESURFER_HOME}/bin:${PATH}

FSLDIR=/usr/local/fsl; export FSLDIR
export PATH=${FSLDIR}/bin:${PATH}
. ${FSLDIR}/etc/fslconf/fsl.sh
source ${FREESURFER_HOME}/SetUpFreeSurfer.sh

# Launch Freeview with the specified images and parameters
freeview -v \
${3}/${2}_FLAIR_coreg_registered_to_${1}.nii.gz \
${3}/${2}_T2_coreg_registered_to_${1}.nii.gz \
${3}/${2}_T1_coreg_registered_to_${1}.nii.gz \
${3}/${2}_CT1_coreg_registered_to_${1}.nii.gz \
${3}/${1}_FLAIR_coreg.nii.gz \
${3}/${1}_T2_coreg.nii.gz \
${3}/${1}_T1_coreg.nii.gz \
${3}/${1}_CT1_coreg.nii.gz \
${3}/segmentation_native_${2}_CT1_coreg_registered_to_${1}.nii.gz:colormap=lut:opacity=0.4 \
${3}/segmentation_native_${1}_CT1_coreg.nii.gz:colormap=lut:opacity=0.4 \
-ras -30 40 18


exam1 (reference): 10_036_20150101
exam2 (moving): 10_036_20160529
REGISTERED_DIR: /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036
Checking paths...
/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20150101_FLAIR_coreg.nii.gz
/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20160529_FLAIR_coreg_registered_to_10_036_20150101.nii.gz
/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20150101_T2_coreg.nii.gz
/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20160529_T2_coreg_registered_to_10_036_20150101.nii.gz
/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20150101_CT1_coreg.nii.gz
/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered/10_036/10_036_20160529_CT1_coreg_registered_to_10_036_20150101.nii.gz
/home/marianne/prj/glioma_recurrence/gliom

INFO: using NIfTI-1 qform 
INFO: using NIfTI-1 qform 
INFO: using NIfTI-1 qform 
INFO: using NIfTI-1 qform 
INFO: using NIfTI-1 qform 
INFO: using NIfTI-1 qform 
INFO: using NIfTI-1 qform 
INFO: using NIfTI-1 qform 
